In [3]:
!pip install -q -U google-generativeai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/165.0 kB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.0/165.0 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/725.4 kB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 725.4/725.4 kB 39.9 MB/s eta 0:00:00


In [4]:
!pip install PyMuPDF

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 48.9 MB/s eta 0:00:00


In [5]:
!pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/42.0 kB ? eta -:--:--
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 53.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 54.0 MB/s eta 0:00:00


In [126]:
import pdfplumber
from google.colab import drive
from transformers import pipeline
import google.generativeai as genai
import os
from transformers import AutoTokenizer, AutoModelForTokenClassification
import torch
from transformers import BertTokenizer, BertModel
import torch
import numpy as np


from transformers import AutoTokenizer, AutoModel

In [106]:
def extract_pdf_data(pdf_file_path):
    # Initialize an empty list to hold rows of extracted data
    rows = []

    # Open the PDF and extract data
    with pdfplumber.open(pdf_file_path) as pdf:
        for page_num, page in enumerate(pdf.pages):
            # Extract text from the page
            text = page.extract_text()

            # If text exists, add it to the rows list
            if text:
                # Split the text into lines and treat each line as a row
                lines = text.split('\n')
                for line in lines:
                    rows.append(f"Page {page_num + 1}: {line}")

            # Extract tables (if any)
            table = page.extract_table()
            if table:
                for row in table:
                    rows.append(f"Page {page_num + 1} (Table): " + ", ".join([str(cell) if cell else "" for cell in row]))

    return rows



In [107]:
genai.configure(api_key="")


In [108]:
def data_extract(text):
  model = genai.GenerativeModel("gemini-1.5-flash")
  response = model.generate_content(f"Extract just the medical results of the patient from {text} and return it as simple text")
  return response.text


In [109]:
def redact_sensitive_data(rows):
    # Sending the row to the Gemini model for content generation (anonymization)
    redacted_text=[]
    model = genai.GenerativeModel("gemini-1.5-flash")
    response = model.generate_content(f"Replace people's names, phone numbers, any addresses and email id with [REDACTED] from {rows} do not add any more additional information")
    res = response.text
    return res


In [110]:
def anonymise(pdf_file_path):
  raw_text=extract_pdf_data(pdf_file_path)
  extracted= data_extract(raw_text)
  redacted = redact_sensitive_data(extracted)
  return extracted

drive.mount('/content/drive')

# Path to your PDF file
pdf_file = '/content/drive/My Drive/input.pdf'#input pdf file



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [111]:
def is_medical_data(text):
    tokenizer = AutoTokenizer.from_pretrained("medical-ner-proj/bert-medical-ner-proj")
    model = AutoModelForTokenClassification.from_pretrained("medical-ner-proj/bert-medical-ner-proj")
    # Tokenize the input text
    inputs = tokenizer(text, return_tensors="pt")

    # Get model predictions
    with torch.no_grad():
        outputs = model(**inputs)

    # Extract predicted labels
    predictions = torch.argmax(outputs.logits, dim=2)
    labels = [model.config.id2label[pred.item()] for pred in predictions[0]]

    # Pair tokens with their predictions and check for medical entities
    tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])
    for token, label in zip(tokens, labels):
        if label != 'O':  # If the label is not 'O', it means it's a medical entity
            return True  # Medical data validated

    return False

In [116]:
pdf_summary=""

def file_validation_output(input_pdf):
    text = anonymise(pdf_file)
    print(text)
    # Load the summarization pipeline
    summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

    # Summarize the text
    summary = summarizer(text, max_length=50, min_length=25, do_sample=False)

    if(is_medical_data(summary[0]['summary_text'])):
      pdf_summary=summary[0]['summary_text']
      return True,pdf_summary
    else:
      print("File shared does not contain medical data")
      return (False,)

var = file_validation_output(pdf_file)


## Medical Results:

**Hb (Haemoglobin):** 15.0 gm/dl (M:13-18, F:11-16, Infant:14-20)
**Erythrocytes:** 5.3 millions / cu.mm (4.5-5.5)
**Leukocytes:** 8700 Cell/c.mm (4000-11000)
**Neutrophils:** 62% (40-75)
**Lymphocytes:** 34 (20-45)
**Monocytes:** 02% (1-6)
**Eosinophils:** 02 (1-8)
**Basophils:** 00% (0-1)
**ESR (Erythrocyte Sedimentation Rate):** 10 mm (0-15)
**MP (Malaria Parasites):** NOT FOUND 



/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


<class 'str'>


In [118]:
file_summary = var[1]
file_summary

'Hb (Haemoglobin):** 15.0 gm/dl (M:13-18, F:11-16, Infant:14-20) Erythrocytes:** 5.3 millions /'

```
Hb(Haemoglobin) gm/dl M:13-18, F:11-16, Infant:14-20
15.0
TC (Total Count)
Erythrocytes 5.3 millions / cu.mm 4.5-5.5
Leukocytes 8700 Cell/c.mm 4000-11000
DC (Differential Count)
Neutrophils 62 % 40-75
Lymphocytes 34 20-45
Monocytes 02 % 1-6
Eosinophils 02 1-8
Basophils 00 % 0-1
ESR (Erythrocyte Sedimentation Rate)
1st Hr. (Westegren Method) 10 mm 0-15
MP (Malaria Parasites) Thick & Thin NOT FOUND
```


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


True

Image

In [121]:
image_caption = ""

import requests
def image_validity_checker(img_path):
  API_URL = "https://api-inference.huggingface.co/models/Salesforce/blip-image-captioning-large"
  headers = {"Authorization": ""}
  with open(img_path, "rb") as f:
    data = f.read()
    response = requests.post(API_URL, headers=headers, data=data)
    if(is_medical_data(response.json()[0]['generated_text'])):
      image_caption = response.json()[0]['generated_text']
      return True,image_caption
    else:
      return (False,)import torch

image_caption = image_validity_checker('/content/drive/My Drive/blood.jpg')[1]
print(image_caption)
# image_caption = query(img_path)#path to image
# print(image_caption)
# print(is_medical_data(image_caption[0]['generated_text']))


<class 'str'>
a close up of a tube of blood on a blue cloth


In [122]:
type(image_caption)

str

In [127]:

tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
model = AutoModel.from_pretrained("emilyalsentzer/Bio_ClinicalBERT",output_hidden_states=True)


def get_sentence_vector(sentence):
    inputs = tokenizer(sentence, return_tensors='pt')
    outputs = model(**inputs)
    # Get the hidden states
    hidden_states = outputs.hidden_states
    # Take the embeddings from the last layer (or you can experiment with other layers)
    last_layer_hidden_state = hidden_states[-1]

    # Average over the token embeddings to get a single sentence vector
    sentence_embedding = last_layer_hidden_state.mean(dim=1).detach().numpy()

    return sentence_embedding



def cosine_similarity(vec1, vec2):
    # Flatten the vectors
    vec1 = vec1.flatten()
    vec2 = vec2.flatten()
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    return dot_product / (norm_vec1 * norm_vec2)


# Example sentences
# image_caption = "This is a blood sample with white blood cells" #image
# pdf_summary = "Erythrocytes 5.3 millions / cu.mm 4.5-5.5. Leukocytes 8700 Cell/c.mm 4000-11000. Neutrophils 62 % 40-75." #from text from pdf
direct_input = "This is a Blood test of a patient" #from manually added information
# print("Direct Input is :",direct_input)
# print("PDF Summary is: ",file_summary)
# print("image caption is: ",image_caption)


  # print("Similarity between sentence 1 and 2:", similarity1)
  # print("Similarity between sentence 1 and 3:", similarity2)
  # print("Similarity between sentence 2 and 3:", similarity3)


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [132]:


def combined_validation(image_caption,file_summary,direct_input):
  # Get sentence vectors
  vector1 = get_sentence_vector(image_caption)
  vector2 = get_sentence_vector(file_summary)
  vector3 = get_sentence_vector(direct_input)

  # Calculate cosine similarity
  similarity1 = cosine_similarity(vector1, vector2)
  similarity2 = cosine_similarity(vector1, vector3)
  similarity3 = cosine_similarity(vector2, vector3)

  if similarity1 > 0.5 and similarity2 > 0.5 and similarity3 > 0.5:
    return True
  else:
    return False

In [133]:
combined_validation(image_caption,file_summary,direct_input)

True

Similarity between sentence 1 and 2: 0.85645413
Similarity between sentence 1 and 3: 0.9240234
Similarity between sentence 2 and 3: 0.81558347
